In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa

import datetime

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

def f_bits2array(bits):
  return [b*2**p for p,b in enumerate(np.flip(bits)) if b!=0]

def f_class_weights(yy):
  yy_labels,yy_counts = np.unique(yy,return_counts=True,axis=0)
  cc_weights = {i:0 for i in range(np.max(yy.ravel())+1)}
  yy_weights = len(yy)/(len(yy_labels)*yy_counts)
  cc_weights.update({yy_labels[i][0]:yy_weights[i] for i,l in enumerate(yy_labels)})
  return cc_weights

def f_savefig(gname,gmetadata,gshow=False):
  plt.savefig(gname,dpi=plt.rcParams["figure.dpi"],metadata=gmetadata,bbox_inches="tight",pad_inches=.2)
  if gshow: plt.show()
  plt.close()  

In [5]:
letters = pd.DataFrame(
    np.array([[0,1,2,4,8,16,32,64,128],['N','R','S','NF','L','ER','EL','D','C']]).T, columns=["number","letter"]
)

letters["number"] = letters["number"].astype(np.int16)

letters

,number,letter
0,0,N
1,1,R
2,2,S
3,4,NF
4,8,L
5,16,ER
6,32,EL
7,64,D
8,128,C


In [2]:
fpath = os.path.join(os.getcwd(),"data")
fname = os.path.join(fpath,"data.1.aug.0k.npz")

fpath = os.path.join(os.getcwd(),"data")
fprefix = "aug.0k"
fversion = "n"
fname = os.path.join(fpath,"data.1.%s.npz"%(fprefix))

fresult = os.path.join(os.getcwd(),"model","wisa.net.%s.%s"%(fprefix,fversion))

print("result:",fresult)

fmetadata = {"Title":fprefix,"Creation Time":datetime.datetime.now().strftime("%d.%m.%Y %H:%M:%S")}

data = np.load(fname)

X = data['arr_0']
y = data['arr_1']

# small anomaly removal
X[X==3] = 2

X_raw = X.copy()
y_raw = y.copy()

types = np.arange(y.sum(axis=1).max()+1)
y = np.packbits(y,axis=1)

y_index = {n:i for i,n in enumerate(np.unique(y))}
y = np.array([[y_index[n]] for n in y.flatten()])

label_index = {y_index[k]:k for k in y_index.keys()}

labels,counts = np.unique(y,return_counts=True,axis=0)
c_weights = f_class_weights(y)

result: c:\Develop\vsc\wisa\develop\model\wisa.net.aug.0k.n


In [3]:
assemblies = []
for t in types:    
  ll = np.unique(y_raw[np.where(y_raw.sum(axis=1)==t)[0]],axis=0)  
  for l in ll:    

    index = np.argwhere(np.packbits(y_raw,axis=1)==np.packbits(l))[:,0]

    assemblies.append(
      dict(
        type=t,label=l,
        number=np.packbits(l)[0],
        number_index=y_index[np.packbits(l)[0]],
        index=index,
        assembly=f_bits2array(l),
        bin=len(index)
      ))

assemblies = pd.DataFrame(assemblies)
assemblies = assemblies.sort_values(by=["type","number"])

In [4]:
size,w,h = X.shape

X_ = X.copy()
y_ = y.copy()

X_ = X_.astype('float32')
X_ = (X_-X_.min())/(X_.max()-X_.min())

X_train,X_test,y_train,y_test = train_test_split(X_,y_,test_size=0.2,random_state=42)
X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,random_state=42)

X_train = X_train.reshape(X_train.shape[0],w,h,1)
X_test = X_test.reshape(X_test.shape[0], w,h, 1)
X_valid = X_valid.reshape(X_valid.shape[0], w,h, 1)

input_shape = (w,h,1)
output_shape = np.max(np.unique(y_train))+1

c_train_weights = f_class_weights(y_train)
c_valid_weights = f_class_weights(y_valid)

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_valid = tf.keras.utils.to_categorical(y_valid)

In [5]:
# median blur to denoise wafer binary maps
import cv2

def f_median_blur(X_,y_):

  ksize = 3

  X_flt = np.zeros_like(X_)
  for k in range(X_.shape[0]):
    X_flt[k,:,:] = cv2.medianBlur(X_[k,:,:],ksize=ksize)

  y_flt = y_.copy()

  X_flt_min,X_flt_max = X_flt.min(),X_flt.max()
  X_flt = (X_flt-X_flt_min)/(X_flt_max-X_flt_min)

  return X_flt,y_flt

X_flt,y_flt = f_median_blur(X_,y_)  

In [6]:
# -----------------------------
#
def f_samples(data,number):
  ss = []  
  nn = assemblies.loc[assemblies["number"]==number,"assembly"].to_numpy()[0]  
  nn = nn if len(nn)>0 else [0]
  for n in nn:
    ii = assemblies.loc[assemblies["number"]==n,"index"].to_numpy()[0]
    k = ii[np.random.randint(low=0,high=ii.shape[0],size=1)]
    I = data[k,:,:]
    I = I.reshape(I.shape[1:])  
    ss.append(dict(sample=I.copy(),raw=I.copy(),index=k,number=n))  
  return ss

# -----------------------------
#
def f_blend(samples,w=52,h=52,overlap=True):  
  I = np.zeros((h,w*samples.shape[0])) if not overlap else np.zeros((h,w))
  for i in range(samples.shape[0]):
    sample = samples.iloc[i,:]
    if not overlap:
      I[:,i*w:(i+1)*w] = sample["sample"]
    else:
      I = np.maximum(I,sample["sample"])  
  return I

# -----------------------------
#
def f_augment(samples,method=['rotate']):  
  frame = pd.DataFrame(samples)
  for i in range(samples.shape[0]):    
    sample = frame.iloc[i,:]
    I = sample["sample"]
    if np.isin(method,['rotate']):
      r = np.pi*np.random.uniform(5,10)/10.
      I = tfa.image.rotate(I,r).numpy()    
    frame.at[i,"sample"] = I  
  return frame

# -----------------------------
#
def f_build(data,filter,number):
  samples = f_samples(data,number=0) + f_samples(filter,number=number)
  samples = pd.DataFrame(samples)
  samples = f_augment(samples)    
  return f_blend(samples,overlap=True)

# -----------------------------
#
def f_compute(bin_max):

  XX = []
  yy = []

  numbers = assemblies["number"].unique()

  for number in numbers:
    
    bin = assemblies.loc[assemblies["number"]==number,"bin"].to_numpy()[0]
    index = assemblies.loc[assemblies["number"]==number,"index"].to_numpy()[0]
    
    dbin = bin_max-bin  

    if dbin < 0:    
      index = index[np.random.randint(0,bin-1,bin_max)]
      print("%03d: %04d < %04d "%(number,bin_max,bin))

    XX = XX + list(X_raw[index,:,:])
    yy = yy + list(y_raw[index])

    if dbin > 0:
    
      for i in range(dbin):

        T = f_build(X_,X_flt,number)  
              
        T[T.round(2)==1.0] = 2
        T[T.round(2)==0.5] = 1        

        T = T.astype(np.uint8)
        
        XX.append(T)
        yy.append(np.unpackbits(np.array([number]).astype(np.uint8)))

      print("%03d: %04d > %04d"%(number,bin,bin_max))

  XX = np.array(XX).astype(np.uint8)
  yy = np.array(yy).astype(np.uint8)

  return XX,yy

# -----------------------------
#
def f_load(fname):
  data = np.load(fname)
  X = data['arr_0']
  y = data['arr_1']
  
  X_raw = X.copy()
  y_raw = y.copy()

  types = np.arange(y.sum(axis=1).max()+1)
  y = np.packbits(y,axis=1)

  y_index = {n:i for i,n in enumerate(np.unique(y))}
  y = np.array([[y_index[n]] for n in y.flatten()])

  label_index = {y_index[k]:k for k in y_index.keys()}

  labels,counts = np.unique(y,return_counts=True,axis=0)
  c_weights = f_class_weights(y)

  return X,y,y_raw,y_index,c_weights,types
  
# -----------------------------
#  
def f_save(fname,XX,yy):
  return np.savez_compressed(fname,XX,yy)

# -----------------------------
#
def f_assemblies(y,y_index,types):
  assemblies = []
  for t in types:    
    ll = np.unique(y[np.where(y.sum(axis=1)==t)[0]],axis=0)  
    for l in ll:    
      index = np.argwhere(np.packbits(y,axis=1)==np.packbits(l))[:,0]
      assemblies.append(
        dict(
          type=t,label=l,
          number=np.packbits(l)[0],
          number_index=y_index[np.packbits(l)[0]],
          index=index,
          assembly=f_bits2array(l),
          bin=len(index)
        ))

  assemblies = pd.DataFrame(assemblies)
  return assemblies.sort_values(by=["type","number"])  

# -----------------------------
#
ffactor = 2
fname = os.path.join(fpath,"data.1.aug.%dk.npz"%(ffactor))

#XX,yy = f_compute(bin_max=1000*ffactor)  
#f_save(fname,XX,yy)


In [7]:
X,y,y_raw,y_index,c_weights,types = f_load(fname)
assemblies = f_assemblies(y_raw,y_index,types)

print(assemblies.loc[:,~assemblies.columns.isin(["index"])].to_markdown())

|    |   type | label             |   number |   number_index | assembly        |   bin |
|---:|-------:|:------------------|---------:|---------------:|:----------------|------:|
|  0 |      0 | [0 0 0 0 0 0 0 0] |        0 |              0 | []              |  2000 |
|  1 |      1 | [0 0 0 0 0 0 0 1] |        1 |              1 | [1]             |  2000 |
|  2 |      1 | [0 0 0 0 0 0 1 0] |        2 |              2 | [2]             |  2000 |
|  3 |      1 | [0 0 0 0 0 1 0 0] |        4 |              3 | [4]             |  2000 |
|  4 |      1 | [0 0 0 0 1 0 0 0] |        8 |              4 | [8]             |  2000 |
|  5 |      1 | [0 0 0 1 0 0 0 0] |       16 |              6 | [16]            |  2000 |
|  6 |      1 | [0 0 1 0 0 0 0 0] |       32 |             10 | [32]            |  2000 |
|  7 |      1 | [0 1 0 0 0 0 0 0] |       64 |             14 | [64]            |  2000 |
|  8 |      1 | [1 0 0 0 0 0 0 0] |      128 |             26 | [128]           |  2000 |
|  9 |    

In [9]:
samples = []

n_indizes = [0,-1]
for t in types:
    view = assemblies[assemblies["type"]==t]
    nn = np.unique(view["number"].to_numpy())    
    for n in nn:
        index = view.loc[view["number"]==n,"index"].to_numpy()[0]

        sample = np.zeros((h*len(n_indizes),w))
        for i,n_index in enumerate(n_indizes):
            I = X[index[n_index],:,:].copy()           
            sample[i*h:(i+1)*h,:] = I

        samples.append(
            dict(
                type=t,
                number=n,
                sample=sample
            )
        )

samples = pd.DataFrame(samples)

# -----------------------------

n_cols = samples.groupby(by=["type"]).agg({"number":"count"}).reset_index()["number"].max()
n_rows = len(types)

plt.rcParams["font.size"] = 8
plt.rcParams["figure.dpi"] = 80

n_width,n_height = 2,2*len(n_indizes)
fig, ax = plt.subplots(nrows=n_rows,ncols=n_cols,figsize=[n_width*n_cols,n_height*n_rows])

for j,t in enumerate(types):
    
    view = samples[samples["type"]==t]
    nn = np.unique(view["number"].to_numpy())
    
    for i,n in enumerate(nn):
        
        assembly = assemblies.loc[(assemblies["type"]==t)&(assemblies["number"]==n),"assembly"].to_numpy()[0]        
        
        sample = view.loc[view["number"]==n,"sample"].to_numpy()[0]
                        
        ax[j,i].imshow(sample[:,:],cmap=plt.cm.Blues)

        title = "%03d %s"%(n,assembly) if i>0 else "%02d:%03d %s"%(t,nn[0],assembly)
        ax[j,i].set_title(title,x=0,ha="left")


for a in ax.ravel():
    a.set_axis_off()

fname = os.path.join(fresult,"%s.%d.png"%(fprefix,ffactor))
f_savefig(fname,fmetadata)

fname

'c:\\Develop\\vsc\\wisa\\develop\\model\\wisa.net.aug.0k.n\\aug.0k.2.png'

In [12]:
X_flt,y_flt = f_median_blur(X,y)

In [35]:
ss = [
    dict(type=0,number=0,index=0,sample=None,image=None,title=""),
    dict(type=1,number=2,index=150,sample=None,image=None,title=""),
    dict(type=1,number=8,index=10,sample=None,image=None,title="")
]

for s in ss:
    i = assemblies.loc[(assemblies["type"]==s["type"])&(assemblies["number"]==s["number"]),"index"].to_numpy()[0][s["index"]]
    s["sample"] = X_flt[i,] if s["type"]!=0 else X[i,]/X[i,].max()
    s["image"] = X[i,] if s["type"]!=0 else None
    s["letter"] = letters.loc[letters["number"]==s["number"],"letter"].to_numpy()[0]
    s["title"] = "%02d:%03d %s"%(s["type"],s["number"],s["letter"])

ss = pd.DataFrame(ss)
ss = f_augment(ss)

A = f_blend(ss,overlap=True)
aa = ss["number"].agg(list)

s = dict(
        type=len(aa)-1,
        number=np.sum(aa),
        index=-1,
        sample=A,
        image=None,
        title=""
    )

s["title"] = "%02d:%03d"%(s["type"],s["number"])

ss = ss.append(pd.DataFrame([s]))

# ---------------------------

n_cmp = [plt.cm.Greys,plt.cm.Blues,plt.cm.Purples,plt.cm.Greens,plt.cm.Oranges]

plt.rcParams["font.size"] = 8
plt.rcParams["figure.dpi"] = 120

n_rows,n_cols = 2,ss.shape[0]
n_width,n_height = 2.5,2.5
fig, ax = plt.subplots(nrows=n_rows,ncols=n_cols,figsize=[n_width*n_cols,n_height*n_rows])

for j in range(ss.shape[0]):

    s = ss.iloc[j,:]

    if s["image"] is not None: ax[0,j].imshow(s["image"],cmap=n_cmp[s["type"]])
    if s["sample"] is not None: ax[1,j].imshow(s["sample"],cmap=n_cmp[s["type"]])

    ax[1,j].set_title("%s"%(s["title"]),x=0,ha="left")

for a in ax.ravel():
    a.set_axis_off()

plt.tight_layout()

fname = os.path.join(fresult,"%s.%d.sample.png"%(fprefix,ffactor))
f_savefig(fname,fmetadata)
fname

'c:\\Develop\\vsc\\wisa\\develop\\model\\wisa.net.aug.0k.n\\aug.0k.2.sample.png'